In [1]:
from datetime import datetime
from datetime import timedelta
import time
from Bio import Entrez
from Bio import Medline
import pandas as pd
from pandas import read_csv
import os
import re
import pickle
import lzma

Entrez.email = os.environ['USEREMAIL']



In [2]:
###############################################################################
## This module takes a list of entrez gene ids, looks up PMIDs associated with
## each gene, obtains the authors of each PMID along with their affiliations
## The returned dataframe can be processed for top authors and more
## Note that Entrez caps requests at 3/second without API key
## With API key, requests are capped at 10/second
## To be on the safe side, this module voluntarily throttles (sleeps) to
## 2 requests/second (sleep is half a second)
###############################################################################
def retrieve_detailed_pubs_by_gene(genelist):
    timestart = datetime.now().time()
    genefailures = []
    pmid_failures = []
    pmid_author_fail = []
    author_df = pd.DataFrame(columns = ["AU", "FullName","AuthorDetails","pmid","publish_date"])
    PublicationDetails = []
    print(timestart, 'obtaining publication details and authors for each gene.')
    for geneid in genelist:
        print ('fetching pmids for: '+str(geneid))
        try: 
            record = Entrez.read(Entrez.elink(dbfrom="gene", id=geneid))
            PMIDList=[] ##creates an empty list to store PMIDs   
            for link in record[0]["LinkSetDb"][0]["Link"] : ##retrieves each PMID stored in the record associated with the gene ID
                PMIDList.append(link["Id"]) ##stores PMID's linked to the gene into the list
            if len(PMIDList) > 30:
                for PMID in PMIDList: #iterates through the PMID list
                    try:
                        handle = Entrez.efetch(db="pubmed", id=int(PMID), rettype="medline", retmode="text")
                        records = Medline.parse(handle) ##parses pubmed entry for that ID and records the author
                        for record in records:
                            try:
                                PublicationDate = record.get("DP","?") #writes the publication date 
                                pubdate_type = "DP"
                            except:
                                PublicationDate = record.get("EDAT","?") #writes the initial Entrez record submission date 
                                pubdate_type = "EDAT"
                            PublicationDetails.append({'geneid':str(geneid),'pmid':PMID,'PublicationDate':PublicationDate,"PubDateType":pubdate_type})
                            try:
                                AuthorSet = record.get("AU","?") #writes the record to a list called AuthorSet                     
                                FullAuthorSet = record.get("FAU","?") #writes the record to a list called AuthorSet
                                AuthorDetails = record.get("AD","?") #writes the record to a list called AuthorDetails
                                tmp_df = pd.DataFrame({'AU':pd.Series(AuthorSet), "FullName":pd.Series(FullAuthorSet),'AuthorDetails': pd.Series(AuthorDetails)})
                                tmp_df['pmid']=PMID
                                tmp_df['publish_date']=PublicationDate
                                author_df = pd.concat((author_df,tmp_df),ignore_index=True)
                            except:
                                pmid_author_fail.append(PMID)
                        time.sleep(0.5)
                    except:
                        #print('bad pmid: ', PMID)
                        pmid_failures.append({'geneid':geneid,'pmid':PMID})
            else:
                genefailures.append(geneid)
        except:
            genefailures.append(geneid)
        
    PublicationDetailsDF = pd.DataFrame(PublicationDetails)
    PMIDfailsDF = pd.DataFrame(pmid_failures)
    timeend = datetime.now().time()
    print(timeend)
    return(PublicationDetailsDF, author_df, genefailures, pmid_author_fail, PMIDfailsDF)


In [3]:
################################################################################
## This uses a list of PMIDs to pull author information
################################################################################

#PMIDList = [23039619,29390967,31363486,30951672] ##Unit test

def retrieve_authors_by_pmids(PMIDList):
    print(datetime.datetime.now().time())
    author_df = pd.DataFrame(columns = ["AU", "FullName","AuthorDetails","pmid","publish_date"])
    PublicationDetails = []
    PMIDFails = []
    for PMID in PMIDList: #iterates through the PMID list
        try:
            #print('fetching authors for: '+str(PMID))
            handle = Entrez.efetch(db="pubmed", id=PMID, rettype="medline", retmode="text")
            records = Medline.parse(handle) ##parses pubmed entry for that ID and records the author
            for record in records:
                AuthorSet = record.get("AU","?") #writes the record to a list called AuthorSet
                FullAuthorSet = record.get("FAU","?") #writes the record to a list called AuthorSet
                try:
                    AuthorDetails = record.get("AD","?") #writes the record to a list called AuthorDetails
                except:
                    AuthorDetails = ['No details']
                try:
                    PublicationDate = record.get("DP","?") #writes the initial Entrez record submission date
                    PublicationDetails.append({'pmid':PMID, 'PubDateType':'PD','PubDate':PublicationDate})
                except:
                    PublicationDate = record.get("EDAT","?")
                    PublicationDetails.append({'pmid':PMID, 'PubDateType':'EDAT','PubDate':PublicationDate})
                #print(len(AuthorSet),len(AuthorDetails),AuthorSet[0],AuthorDetails[0])
                tmp_df = pd.DataFrame({'AU':pd.Series(AuthorSet), "FullName":pd.Series(FullAuthorSet),'AuthorDetails': pd.Series(AuthorDetails)})
                tmp_df['pmid']=PMID
                tmp_df['PubDate']=PublicationDate
                author_df = pd.concat((author_df,tmp_df))
        except:
            PMIDFails.append(PMID)
            print("pmid not found: ",PMID)

    PublicationDF = pd.DataFrame(PublicationDetails)
    print(datetime.datetime.now().time())
    return(PublicationDF,author_df,PMIDFails)



In [4]:
################################################################################
## This module parses email addresses from the AD/Affiliation field if available
## input is the author dataframe resulting from this module:
## retrieve_detailed_pubs_by_gene
## Note that Articles prior to: October 1, 2013, will have an affiliation ONLY 
## for the first author. Details on that here: 
## https://www.nlm.nih.gov/pubs/techbull/so13/brief/so13_author_affiliations.html
################################################################################

def parse_out_emails(author_df):
    author_df.reset_index(inplace=True)
    try:
        author_df.drop(['level_0'],axis=1,inplace=True)
    except Exception: 
        pass
    try:
        author_df.drop(['index'],axis=1,inplace=True)
    except Exception: 
        pass
    author_df['email'] = author_df['AuthorDetails'].str.extract(r'([^@|\s]+@[^@]+\.[^@|\s]+)')
    author_emails = author_df.loc[author_df['email'].notnull()]
    author_emails_dots = author_emails.loc[author_emails['email'].str[-1]=="."]
    author_emails_dots['email'] = author_emails_dots['email'].str[:-1]
    author_emails.update(author_emails_dots)
    author_df.update(author_emails)
    return(author_df)


In [5]:
###############################################################################
## This module takes the resulting dataframes from the previous module
## merges them and then gets a count of the number of publications an author
## has for each gene. Note that the optional parameter, method allows for the
## counts to be based on the author "AU" or full name of the author "FullName"
## The default is the FullName
###############################################################################
def get_top_authors_from_dfs(PublicationDetailsDF, author_df, method="FullName"):
    all_merged_df = author_df_deets.merge(PublicationDetailsDF, on='pmid',how='left')
    all_merged_df.drop_duplicates(keep='first',inplace=True)
    top_authors_per_gene = all_merged_df.groupby(['geneid',method,'FullName']).size().reset_index(name='pubcounts')
    top_authors_per_gene.sort_values(by=['geneid','pubcounts'],ascending=[True,False],inplace=True)
    return(top_authors_per_gene)

In [6]:
## Pull up all pmids per genes, get author details, pull out available email addresses
## Perform groupby counts to get top contributing authors per gene
        
def get_authors(genelist,datapath,test=False):
    if test==True:
        genelist = [439921,55768] ## for unit test     
    PublicationDetailsDF, author_df, genefailures, pmid_author_fail, PMIDfailsDF = retrieve_detailed_pubs_by_gene(genelist)
    author_df_deets = parse_out_emails(author_df)
    PublicationDetailsDF.to_csv(os.path.join(datapath,'PublicationDetailsDF.tsv'),sep='\t',header=True)
    #author_df.to_csv(os.path.join(datapath,'author_df.tsv'),sep='\t',header=True)
    author_pickle = pickle.dumps(author_df)
    with lzma.open(os.path.join(datapath,"author_df.xz"), "w") as f:
        f.write(author_pickle)
    #author_df_deets.to_csv(os.path.join(datapath,'author_df_deets.tsv'),sep='\t',header=True)
    PMIDfailsDF.to_csv(os.path.join(datapath,'PMIDfailsDF.tsv'),sep='\t',header=True)
    with open(datapath+'genefailures.txt','w') as outwrite:
        for eachgene in genefailures:
            outwrite.write(str(eachgene)+'\n')
        outwrite.close()

def deal_with_failures(datapath,hasfailures = True):
    oldPublicationDetailsDF = read_csv(os.path.join(datapath,'PublicationDetailsDF.tsv'),delimiter='\t',header=0,index_col=0)
    oldauthor_df = read_csv(os.path.join(datapath,'author_df.tsv'),delimiter='\t',header=0,index_col=0)
    oldauthor_df_deets = read_csv(os.path.join(datapath,'author_df_deets.tsv'),delimiter='\t',header=0,index_col=0)
    oldPMIDfailsDF = read_csv(os.path.join(datapath,'PMIDfailsDF.tsv'),delimiter='\t',header=0,index_col=0)
    i=0
    while hasfailures == True:
        with open(datapath+'genefailures.txt','r') as infile:
            tmpgenelist = []
            for line in infile:
                tmpgenelist.append(line.strip())
            infile.close()
        tmpPublicationDetailsDF, tmpauthor_df, tmpgenefailures, tmppmid_author_fail, tmpPMIDfailsDF = retrieve_detailed_pubs_by_gene(tmpgenelist)
        tmpauthor_df_deets = parse_out_emails(author_df)
        PublicationDetailsDF = pd.concat((oldPublicationDetailsDF,tmpPublicationDetailsDF),ignore_index=True)
        PublicationDetailsDF.to_csv(os.path.join(datapath,'PublicationDetailsDF.tsv'),sep='\t',header=True)
        author_df = pd.concat((oldauthor_df,tmpauthor_df),ignore_index=True)
        author_df.to_csv(os.path.join(datapath,'author_df.tsv'),sep='\t',header=True)
        author_df_deets = pd.concat((oldauthor_df_deets,tmpauthor_df_deets),ignore_index=True)
        author_df_deets.to_csv(os.path.join(datapath,'author_df_deets.tsv'),sep='\t',header=True)
        PMIDfailsDF = pd.concat((oldPMIDfailsDF,tmpPMIDfailsDF),ignore_index=True)
        PMIDfailsDF.to_csv(os.path.join(datapath,'PMIDfailsDF.tsv'),sep='\t',header=True)
        with open(datapath+'genefailures.txt','w') as outwrite:
            for eachgene in tmpgenefailures:
                outwrite.write(str(eachgene)+'\n')
            outwrite.close()
        if len(tmpgenefailures) < 2:
            hasfailures = False
        i=i+1
    return(i)

# Main Script

### Obtaining Publication and author details

In [7]:
## Pull top authors for high priority genes

datapath = 'data/'
resultpath = 'results/'
genefile = 'priority_by_size.tsv'

prioritylist = read_csv(os.path.join(resultpath,genefile),delimiter='\t',header=0,index_col=0)
genelist = prioritylist['geneID'].unique().tolist()


In [8]:
%%time
get_authors(genelist,datapath,test=False)

08:19:06.190562 obtaining publication details and authors for each gene.
fetching pmids for: 83986
fetching pmids for: 10578
fetching pmids for: 120425
fetching pmids for: 10858
fetching pmids for: 3001
fetching pmids for: 820
fetching pmids for: 57817
fetching pmids for: 4357
fetching pmids for: 10381
fetching pmids for: 51738
fetching pmids for: 4521
fetching pmids for: 54884
fetching pmids for: 1806
fetching pmids for: 100130274
fetching pmids for: 441478
fetching pmids for: 84901
fetching pmids for: 55163
fetching pmids for: 9477
fetching pmids for: 84851
fetching pmids for: 7551
fetching pmids for: 192666
fetching pmids for: 390792
fetching pmids for: 140807
fetching pmids for: 54221
fetching pmids for: 80235
fetching pmids for: 3886
fetching pmids for: 8687
fetching pmids for: 342574
fetching pmids for: 147183
fetching pmids for: 162605
fetching pmids for: 5152
fetching pmids for: 51409
fetching pmids for: 23251
fetching pmids for: 85449
fetching pmids for: 729447
fetching pmids 

fetching pmids for: 79750
fetching pmids for: 54955
fetching pmids for: 80816
fetching pmids for: 84502
fetching pmids for: 140886
fetching pmids for: 57118
fetching pmids for: 246184
fetching pmids for: 79697
fetching pmids for: 114971
fetching pmids for: 80149
fetching pmids for: 125704
fetching pmids for: 389840
fetching pmids for: 6039
fetching pmids for: 1439
fetching pmids for: 56649
fetching pmids for: 5452
fetching pmids for: 55247
fetching pmids for: 6433
fetching pmids for: 344838
fetching pmids for: 344167
fetching pmids for: 9751
fetching pmids for: 8748
fetching pmids for: 8034
fetching pmids for: 1212
fetching pmids for: 9214
fetching pmids for: 112937
fetching pmids for: 84920
fetching pmids for: 27115
fetching pmids for: 51166
fetching pmids for: 7107
fetching pmids for: 114799
fetching pmids for: 122809
fetching pmids for: 387032
fetching pmids for: 140691
fetching pmids for: 493869
fetching pmids for: 23015
fetching pmids for: 10079
fetching pmids for: 319101
fetching

fetching pmids for: 441639
fetching pmids for: 391190
fetching pmids for: 84969
fetching pmids for: 283849
fetching pmids for: 219429
fetching pmids for: 219960
fetching pmids for: 390142
fetching pmids for: 120796
fetching pmids for: 390093
fetching pmids for: 390433
fetching pmids for: 390083
fetching pmids for: 390038
fetching pmids for: 119679
fetching pmids for: 390075
fetching pmids for: 390072
fetching pmids for: 119695
fetching pmids for: 390037
fetching pmids for: 219965
fetching pmids for: 219870
fetching pmids for: 390036
fetching pmids for: 219869
fetching pmids for: 26683
fetching pmids for: 341276
fetching pmids for: 391211
fetching pmids for: 116931
fetching pmids for: 100381270
fetching pmids for: 10344
fetching pmids for: 64577
fetching pmids for: 8837
fetching pmids for: 103910
fetching pmids for: 441608
fetching pmids for: 401665
fetching pmids for: 56655
fetching pmids for: 9832
fetching pmids for: 57721
fetching pmids for: 57654
fetching pmids for: 125963
fetching 

fetching pmids for: 971
fetching pmids for: 55122
fetching pmids for: 641654
fetching pmids for: 56941
fetching pmids for: 1551
fetching pmids for: 54576
fetching pmids for: 124538
fetching pmids for: 390197
fetching pmids for: 57053
fetching pmids for: 146183
fetching pmids for: 23416
fetching pmids for: 387509
fetching pmids for: 119682
fetching pmids for: 56996
fetching pmids for: 390261
fetching pmids for: 10936
fetching pmids for: 54097
fetching pmids for: 64859
fetching pmids for: 127068
fetching pmids for: 399664
fetching pmids for: 347732
fetching pmids for: 10412
fetching pmids for: 138803
fetching pmids for: 138799
fetching pmids for: 255743
fetching pmids for: 79190
fetching pmids for: 442038
fetching pmids for: 10486
fetching pmids for: 388531
fetching pmids for: 286464
fetching pmids for: 83550
fetching pmids for: 64756
fetching pmids for: 26219
fetching pmids for: 387601
fetching pmids for: 343169
fetching pmids for: 255928
fetching pmids for: 166379
fetching pmids for: 2

fetching pmids for: 3742
fetching pmids for: 4849
fetching pmids for: 10795
fetching pmids for: 11018
fetching pmids for: 10641
fetching pmids for: 92597
fetching pmids for: 60676
fetching pmids for: 7484
fetching pmids for: 83903
fetching pmids for: 200959
fetching pmids for: 80778
fetching pmids for: 51032
fetching pmids for: 3443
fetching pmids for: 83692
fetching pmids for: 7554
fetching pmids for: 55520
fetching pmids for: 138804
fetching pmids for: 57560
fetching pmids for: 340485
fetching pmids for: 64062
fetching pmids for: 147841
fetching pmids for: 51117
fetching pmids for: 11248
fetching pmids for: 11264
fetching pmids for: 92906
fetching pmids for: 63970
fetching pmids for: 25890
fetching pmids for: 23387
fetching pmids for: 9899
fetching pmids for: 3743
fetching pmids for: 26476
fetching pmids for: 57176
fetching pmids for: 54825
fetching pmids for: 207063
fetching pmids for: 113263
fetching pmids for: 161357
fetching pmids for: 54780
fetching pmids for: 139135
fetching pm

...

fetching pmids for: 79132
fetching pmids for: 6002
fetching pmids for: 29992
fetching pmids for: 2036
fetching pmids for: 3592
fetching pmids for: 5890
fetching pmids for: 6045
fetching pmids for: 10749
fetching pmids for: 1180
fetching pmids for: 56673
fetching pmids for: 3799
fetching pmids for: 4610
fetching pmids for: 79020
fetching pmids for: 11011
fetching pmids for: 4218
fetching pmids for: 6710
fetching pmids for: 7385
fetching pmids for: 211
fetching pmids for: 863
fetching pmids for: 3036
fetching pmids for: 823
fetching pmids for: 5031
fetching pmids for: 9044
fetching pmids for: 54345
fetching pmids for: 3992
fetching pmids for: 3191
fetching pmids for: 10661
fetching pmids for: 5441
fetching pmids for: 3209
fetching pmids for: 5435
fetching pmids for: 10659
fetching pmids for: 2314
fetching pmids for: 9294
fetching pmids for: 3767
fetching pmids for: 2558
fetching pmids for: 54550
fetching pmids for: 57609
fetching pmids for: 27183
fetching pmids for: 2560
fetching pmids f

fetching pmids for: 8605
fetching pmids for: 6447
fetching pmids for: 284001
fetching pmids for: 57761
fetching pmids for: 1137
fetching pmids for: 9276
fetching pmids for: 8668
fetching pmids for: 171024
fetching pmids for: 5473
fetching pmids for: 6498
fetching pmids for: 9568
fetching pmids for: 5350
fetching pmids for: 130814
fetching pmids for: 774
fetching pmids for: 2069
fetching pmids for: 10152
fetching pmids for: 56288
fetching pmids for: 1404
fetching pmids for: 2488
fetching pmids for: 3547
fetching pmids for: 51138
fetching pmids for: 2941
fetching pmids for: 10267
fetching pmids for: 23181
fetching pmids for: 6833
fetching pmids for: 1174
fetching pmids for: 7021
fetching pmids for: 7903
fetching pmids for: 1478
fetching pmids for: 6448
fetching pmids for: 6690
fetching pmids for: 11057
fetching pmids for: 3646
fetching pmids for: 9328
fetching pmids for: 26277
fetching pmids for: 54332
fetching pmids for: 390
fetching pmids for: 4134
fetching pmids for: 60529
fetching pm

fetching pmids for: 1207
fetching pmids for: 3449
fetching pmids for: 5434
fetching pmids for: 55177
fetching pmids for: 54386
fetching pmids for: 1009
fetching pmids for: 1848
fetching pmids for: 80224
fetching pmids for: 23192
fetching pmids for: 2027
fetching pmids for: 160364
fetching pmids for: 6890
fetching pmids for: 63895
fetching pmids for: 6781
fetching pmids for: 9135
fetching pmids for: 3822
fetching pmids for: 81704
fetching pmids for: 10254
fetching pmids for: 490
fetching pmids for: 6785
fetching pmids for: 6414
fetching pmids for: 79742
fetching pmids for: 2782
fetching pmids for: 84433
fetching pmids for: 5947
fetching pmids for: 11267
fetching pmids for: 6837
fetching pmids for: 265
fetching pmids for: 2110
fetching pmids for: 10681
fetching pmids for: 1725
fetching pmids for: 7799
fetching pmids for: 3746
fetching pmids for: 25842
fetching pmids for: 4705
fetching pmids for: 1617
fetching pmids for: 114757
fetching pmids for: 2533
fetching pmids for: 6634
fetching pm

<ipython-input-4-c796b27a7ce0>:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  author_emails_dots['email'] = author_emails_dots['email'].str[:-1]
C:\Users\gtsueng\Anaconda3\envs\pywikibot\lib\site-packages\pandas\core\frame.py:6386: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[col] = expressions.where(mask, this, that)


Wall time: 8d 5h 51min 12s


In [ ]:
%%time
i = deal_with_failures(datapath,hasfailures = True)

In [ ]:
## Try to pull the information for the PMID failures
print(len(PMIDfailsDF))
print(PMIDfailsDF.head(n=2))
PMIDFailList = PMIDfailsDF['pmid'].unique().tolist()

PublicationDF,authordf,PMIDFails = retrieve_authors_by_pmids(PMIDFailList)
print(len(PMIDFails))

In [ ]:
print(PublicationDetailsDF.head(n=2))
#print(PublicationDF.head(n=2))

PublicationDetailsDF_leftovers = PublicationDF.merge(PMIDfailsDF,on='pmid',how='left')
PublicationDetailsDF_leftovers.drop('Unnamed: 0',axis=1,inplace=True)
print(PublicationDetailsDF_leftovers.head(n=2))

authordf_deets = parse_out_emails(authordf)

In [ ]:
PublicationDetailsDF_leftovers.to_csv(exppath+'PublicationDF_run6.tsv',sep='\t',header=True)
authordf.to_csv(exppath+'author_df_run6.tsv',sep='\t',header=True)
authordf_deets.to_csv(exppath+'author_df_deets_run6.tsv',sep='\t',header=True)
PMIDfailsDF.to_csv(exppath+'PMIDfailsDF_run6.tsv',sep='\t',header=True)

In [ ]:
## get author counts for each gene
top_authors_per_gene = get_top_authors_from_dfs(PublicationDetailsDF, author_df, "FullName")
print(top_authors_per_gene.head(n=3))

top_authors_per_gene.to_csv(exppath+'top_authors_per_gene_FullName.tsv',sep='\t',header=True)

top_authors_per_gene = get_top_authors_from_dfs(PublicationDetailsDF, author_df, "AU")
print(top_authors_per_gene.head(n=3))

top_authors_per_gene.to_csv(exppath+'top_authors_per_gene_AU.tsv',sep='\t',header=True)

## Summarizing and making sense of the results

In [ ]:
author_df = read_csv(exppath+'author_df_run1.tsv', delimiter='\t', header=0)
PublicationDetailsDF = read_csv(exppath+'PublicationDetailsDF_run1.tsv', delimiter='\t', header=0)

i=2
while i < 7:
    tmpdf = read_csv(exppath+"author_df_run"+str(i)+".tsv", delimiter='\t', header=0)
    author_df = pd.concat((author_df,tmpdf), ignore_index=True)
    tmpdeets_df = read_csv(exppath+"PublicationDetailsDF_run"+str(i)+".tsv", delimiter='\t', header=0)
    PublicationDetailsDF = pd.concat((PublicationDetailsDF,tmpdeets_df), ignore_index=True)
    i=i+1

author_df.drop('Unnamed: 0',axis=1,inplace=True)
PublicationDetailsDF.drop('Unnamed: 0',axis=1,inplace=True)

In [ ]:
print(len(PublicationDetailsDF['geneid'].unique()))

In [ ]:
PublicationDetailsDF.rename(columns={'PublicationDate':'publish_date'},inplace=True)
PublicationDetailsDF['geneid'] = PublicationDetailsDF['geneid'].astype(str)
print(len(PublicationDetailsDF), PublicationDetailsDF.head(n=2))
print(len(author_df), author_df.head(n=2))

In [ ]:
all_results = author_df.merge(PublicationDetailsDF,on=['pmid','PubDate','publish_date'],how='left')
print(len(all_results))
all_results.drop_duplicates(keep='first',inplace=True)
print(len(all_results))
print(all_results.head(n=2))

In [ ]:
## sort the authors to get the top authors
author_numbers = all_results.groupby(['geneid','FullName','AU']).size().reset_index(name='author_count')
author_numbers.sort_values(['geneid','author_count'],ascending=[True,False],inplace=True)
print(author_numbers.head(n=2))

In [ ]:
## Get the author email list (wherever email is not nan) and drop any immediate and obvious duplicate entries
author_emails = author_df.loc[author_df['email'].notnull()].sort_values('AU',ascending=True)
author_emails.drop_duplicates(keep='first',inplace=True)
author_emails.sort_values(['FullName','publish_date'],ascending=[True,True],inplace=True)

## fill in publication dates where missing and delete duplicate column
author_emails['publish_date'].loc[author_emails['publish_date'].isnull()] = author_emails['PubDate']
author_emails.drop('PubDate',axis=1,inplace=True)

## Drop duplicates again
author_emails.drop_duplicates(keep='first',inplace=True)

## sort by pmid (assuming pmid numbers only go up over time), de-duplicate to get most recent
author_emails.sort_values(['AU','FullName','pmid'],ascending=[True,True,True],inplace=True)

most_recent_email = author_emails.drop_duplicates(subset=['AU','FullName'],keep='last',inplace=False)
most_recent_email['pmid'] = most_recent_email['pmid'].astype(str)
print(most_recent_email.head(n=2))



In [ ]:
## get most recent author details and merge with most recent author email list

author_no_emails = author_df.loc[author_df['email'].isnull()].sort_values(['AU','FullName','pmid'],ascending=[True,True,True])
author_no_emails.drop_duplicates(subset=['AU','FullName'],keep='last',inplace=True)

author_details = pd.concat((author_no_emails,most_recent_email))                                                                                                           

In [ ]:
top_authors = author_numbers.merge(author_details,on=['AU','FullName'],how='left').fillna('Not Available')
print(len(top_authors))
top_authors.drop_duplicates(keep='first',inplace=True)
print(len(top_authors))
print(top_authors.head(n=2))

In [ ]:
top_authors.to_csv(exppath+'top_ad_author_details.tsv',sep='\t',header=True)

In [ ]:
#### Export the top 20 authors for each gene since exporting ALL the authors is too many

top_authors = read_csv(exppath+'top_ad_author_details.tsv',delimiter='\t',header=0)
top_authors.drop('Unnamed: 0',axis=1,inplace=True)
top_authors.reset_index(inplace=True)
top_authors.rename(columns={'index':'rowid'},inplace=True)
print(top_authors.head(n=2))
top_20_authors = top_authors.groupby('geneid')['author_count'].nlargest(20).reset_index()
top_20_authors.rename(columns={'level_1':'rowid'},inplace=True)

top_20_author_deets = top_20_authors.merge(top_authors,on=['geneid','rowid','author_count'],how='left')
print(len(top_authors), len(top_20_authors), len(top_20_author_deets))
print(top_20_author_deets.head(n=11))
top_20_author_deets.to_csv(exppath+'top_20_author_deets.tsv',sep='\t',header=True)